![Workshop Image](static/image.png)

<div align="center">

# Workshop Ubucon Latinoamerica 2024

</div>

Este es el repositorio para el taller de Ubucon Latinoamerica 2024 titulado:

📊 Crea tu propio laboratorio de Big Data con MicroK8s y Spark

A continuación se encuentra el enlace al video de la presentación [aquí](https://www.youtube.com/watch?v=kSlLjAXISA8):

Recomendamos ver el video de la presentación antes de iniciar el taller.

## Requisitos

**Disclaimer**: Este proyecto no ejecuta en Windows.

Se recomienda correr el taller en una máquina con Ubuntu 22.04 o superior. Y adicionamente instalar el paquete de Snap. Para instalar Snap en tu máquina puedes seguir este enlace:
[Instalar Snap](https://snapcraft.io/docs/installing-snapd)

Para verificar que tienes instalado Snap en tu máquina puedes ejecutar el siguiente comando:
```bash
snap --version
```

## Inicializar el proyecto

1. Ejecute el archivo `setup.sh` para desplegar los componentes de Kubernetes.


In [ ]:
import getpass
import os

password = getpass.getpass()
command = "sudo -S bash setup.sh" #can be any command but don't forget -S as it enables input from stdin
os.system('echo %s | %s' % (password, command))


2. Ejecute el archivo `init_project.sh` para poder configurar los buckets de S3 en MinIO y configurar spark.

In [ ]:
!bash init_project.sh

3. Cargar códigos de prueba, con el script `upload_artifacts.sh`

In [ ]:
!bash upload_artifacts.sh

3. Cargar códigos de prueba, con el script `upload_data.sh`

In [ ]:
!bash upload_data.sh

## Prueba

spark-client.pyspark \
  --username spark \
  --namespace spark \
  --packages org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.0

---

In [ ]:
!spark-client.spark-submit \
    --username spark --namespace spark \
    --deploy-mode cluster \
    s3a://artifacts/python/count_vowels.py

pod_name=$(kubectl get pods -n spark | grep 'count-vowels-.*-driver' | tail -n 1 | cut -d' ' -f1)

# View only the line containing the output
kubectl logs $pod_name -n spark | grep "The number of vowels in the string is"

---

In [ ]:
!spark-client.spark-submit \
    --username spark --namespace spark \
    --deploy-mode cluster \
    s3a://artifacts/python/process_gh_archive_data.py \
    --source_files_pattern=s3a://raw/gh_archive/year=2024/month=01/day=01/hour=0 \
    --destination_files_pattern=s3a://curated/parquet/gh_archive

In [ ]:
!spark-client.spark-submit \
    --username spark --namespace spark \
    --deploy-mode cluster \
    s3a://artifacts/python/process_gh_archive_data.py \
    --source_files_pattern=s3a://raw/gh_archive/year=2024/month=01/day=01/hour=1 \
    --destination_files_pattern=s3a://curated/parquet/gh_archive

---

In [ ]:
!spark-client.spark-submit \
    --username spark --namespace spark \
    --deploy-mode cluster \
    --packages org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.0 \
    s3a://artifacts/python/demo_iceberg.py

---

In [ ]:
!spark-client.spark-submit \
    --username spark --namespace spark \
    --deploy-mode cluster \
    s3a://artifacts/python/process_gh_archive_data_iceberg.py \
    --source_files_pattern=s3a://raw/gh_archive/year=2024/month=01/day=02


---

In [ ]:
!spark-client.spark-submit \
    --username spark --namespace spark \
    --deploy-mode cluster \
    --packages org.apache.hudi:hudi-spark3.4-bundle_2.12:0.15.0  \
    s3a://artifacts/python/process_gh_archive_data_hudi.py  \
    --source_files_pattern=s3a://raw/gh_archive/year=2024/month=01/day=02/hour=0


